In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/NLP

/content/gdrive/MyDrive/NLP


In [3]:
!pip install import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import import_ipynb

from nlp_Preproc_final import preproc
from nlp_model_final import get_model, get_model_with_params, BertModelInitialization
import random
import numpy as np
import pandas as pd
import torch
from tqdm.notebook import tqdm
import time

importing Jupyter notebook from nlp_Preproc_final.ipynb
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 967 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 29.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595729 sha256=6ad60e09ae76c9e4dc50ade8fb2e447057b862fad75352dfa6d1a409a19edadd
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 30.0 MB/s 
Looking 

In [5]:
# 정확도 계산 함수
def accuracy(preds, labels):
    f_pred = np.argmax(preds, axis=1).flatten()
    f_labels = labels.flatten()
    return np.sum(f_pred == f_labels) / len(f_labels)

In [6]:
# 재현을 위해 랜덤시드 고정
seed_val = 2022

**잠깐 ✔ 랜덤시드 고정이란 무엇인가?**
> 학습된 모델의 결과를 동일하게 재현(Reproduction)하는 것은 여러가지 상황에서 팔요하다.  
> 모델을 돌릴 때마다 결과가 달라지지 않도록 고정하는 것이다.

- 수상자가 되어 코드의 정합성을 검증 받게 될 경우,

- 경진대회 참가 도중 팀을 이루어 결과를 공유해야 되는 경우,

- 논문을 작성하여 그 결과를 Reproduction 해야하는 경우 등 여러 상황에서 필요하다.

참고 자료:
https://dacon.io/codeshare/2363
https://pytorch.org/docs/stable/notes/randomness.html


In [7]:
# 랜덤하게 데이터를 추출하기 위한 seed 값 설정
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [8]:
from nlp_tokenization import KoBertTokenizer

# 전체 데이터를 불러오자.
whole_dataset = pd.read_excel('/content/gdrive/MyDrive/NLP/data/chat_data.xlsx')

# KoBERTTokenizer를 불러오자.
tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")

train_dataloader, validation_dataloader = preproc(tokenizer, whole_dataset)

Downloading:   0%|          | 0.00/371k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

length of whole_data : 34388
length of train_data : 30949


nlp_Preproc_final.ipynb:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


In [9]:
# BertModel 생성해서 GPU 혹은 CPU에 등록
# 기존 Device에 등록된 BertModel은 초기화되니, 유의하여 사용할 것.
# 한 번만 실행하고, 그 이후로는 사용하지 않도록 조심!
BertModelInitialization()

Downloading:   0%|          | 0.00/426 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [10]:
# GPU 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model, optimizer, scheduler, epochs, criterion = get_model_with_params(len(train_dataloader), device, torch.cuda.is_available())

# 그래디언트 초기화
model.zero_grad()

In [11]:
# 에폭만큼 반복
for epoch_i in range(epochs):
    print("")
    print('========{:}번째 Epoch / 전체 {:}회 ========'.format(epoch_i + 1, epochs))
    print('훈련 중')

    t0 = time.time() # 시작 시간 설정
    total_loss = 0 # 로스 초기화
    sum_loss = 0
    model.train()  # 훈련모드로 변경
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(tqdm(train_dataloader)):
        
        if step % 50 == 0:
          print("{}번째 까지의 평균 loss : {}".format(step, sum_loss/50))
          sum_loss = 0

        batch = tuple(t.to(device) for t in batch)   # 배치를 GPU에 넣음
        b_input_ids, b_input_mask, b_labels = batch  # 배치에서 데이터 추출

        # nlp_model_final.ipynb 파일에서 BertClassifier 클래스를 정의하고 해당 클래스를 활용하여 'model'을 생성한 바 있다.
        # 위의 코드에서 정의한 배치 데이터를 'model'의 입력 변수로 이용하여, 배치에 대한 forward를 수행해보자.   
        # Hint: BertClassifier의 forward를 수행하기 위해서는 input_ids, attention_mask 변수를 입력받아야 한다. 
        ## 여기에 코드 작성 
        outputs = model(input_ids = b_input_ids.clone().detach(), attention_mask = b_input_mask.clone().detach())
        # 위에서 산출된 7차원 벡터와 criterion, b_labels를 활용해서 loss를 계산해보자.
        ## 여기에 코드 작성
        loss = criterion(outputs, b_labels)

        total_loss += loss.item() # 총 로스 계산
        sum_loss += loss.item()

        loss.backward() # Backward 수행으로 그래디언트 계산
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # 그래디언트 클리핑
        optimizer.step() # 그래디언트를 통해 가중치 파라미터 업데이트
        scheduler.step()  # 스케줄러로 학습률 감소
        model.zero_grad() # 그래디언트 초기화

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            
    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    
    #### 검증 ####
    
    print("")
    print("검증 중")

    t0 = time.time() #시작 시간
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():
            # 위 train과 같은 논리. forward를 수행해서 7차원 벡터 출력    
            # outputs = ~~ 과 같이 작성
            ## 여기에 코드 작성 
             outputs = model(input_ids = b_input_ids.clone().detach(), attention_mask = b_input_mask.clone().detach())
        # CPU로 데이터 이동
        outputs = outputs.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 위에 구현한 accuracy 함수를 활용해서 출력 로짓과 라벨을 비교하여 정확도 계산
        # tmp_eval_accuracy = ~~ 와 같이 작성
        ## 여기에 코드 작성
        tmp_eval_accuracy = accuracy(outputs, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))

# 학습된 모델을 해당 PATH에 저장
PATH = "model.pt"
torch.save(model.state_dict(), PATH)

print("")
print("Training complete!")


========1번째 Epoch / 전체 3회 ========
훈련 중


  0%|          | 0/968 [00:00<?, ?it/s]

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 1.9245180249214173
100번째 까지의 평균 loss : 1.8475653743743896
150번째 까지의 평균 loss : 1.765328552722931
200번째 까지의 평균 loss : 1.6291385507583618
250번째 까지의 평균 loss : 1.5424154686927796
300번째 까지의 평균 loss : 1.5451444268226624
350번째 까지의 평균 loss : 1.4740794610977173
400번째 까지의 평균 loss : 1.4166680598258972
450번째 까지의 평균 loss : 1.4260687732696533
500번째 까지의 평균 loss : 1.3808755278587341
550번째 까지의 평균 loss : 1.3940716695785522
600번째 까지의 평균 loss : 1.3209663915634156
650번째 까지의 평균 loss : 1.366975522041321
700번째 까지의 평균 loss : 1.3107845067977906
750번째 까지의 평균 loss : 1.3148617815971375
800번째 까지의 평균 loss : 1.3109255886077882
850번째 까지의 평균 loss : 1.3119937181472778
900번째 까지의 평균 loss : 1.3080561137199402
950번째 까지의 평균 loss : 1.2781241071224212

  Average training loss: 1.46

검증 중
  Accuracy: 0.5191

========2번째 Epoch / 전체 3회 ========
훈련 중


  0%|          | 0/968 [00:00<?, ?it/s]

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 1.2287184286117554
100번째 까지의 평균 loss : 1.235097050666809
150번째 까지의 평균 loss : 1.2089944434165956
200번째 까지의 평균 loss : 1.1810603976249694
250번째 까지의 평균 loss : 1.1944048178195954
300번째 까지의 평균 loss : 1.1724901366233826
350번째 까지의 평균 loss : 1.1876466476917267
400번째 까지의 평균 loss : 1.190601716041565
450번째 까지의 평균 loss : 1.1870507061481477
500번째 까지의 평균 loss : 1.1782178044319154
550번째 까지의 평균 loss : 1.2000221705436707
600번째 까지의 평균 loss : 1.194103752374649
650번째 까지의 평균 loss : 1.1760064911842347
700번째 까지의 평균 loss : 1.13037992477417
750번째 까지의 평균 loss : 1.1901062250137329
800번째 까지의 평균 loss : 1.1835745894908904
850번째 까지의 평균 loss : 1.1475304901599883
900번째 까지의 평균 loss : 1.182100509405136
950번째 까지의 평균 loss : 1.1564667546749114

  Average training loss: 1.18

검증 중
  Accuracy: 0.5416

========3번째 Epoch / 전체 3회 ========
훈련 중


  0%|          | 0/968 [00:00<?, ?it/s]

0번째 까지의 평균 loss : 0.0
50번째 까지의 평균 loss : 1.1344888722896576
100번째 까지의 평균 loss : 1.0556963384151459
150번째 까지의 평균 loss : 1.0958421158790588
200번째 까지의 평균 loss : 1.099804561138153
250번째 까지의 평균 loss : 1.0445296001434325
300번째 까지의 평균 loss : 1.118138338327408
350번째 까지의 평균 loss : 1.0685707700252534
400번째 까지의 평균 loss : 1.121103092432022
450번째 까지의 평균 loss : 1.0842430698871612
500번째 까지의 평균 loss : 1.072547026872635
550번째 까지의 평균 loss : 1.093722449541092
600번째 까지의 평균 loss : 1.0417646753787995
650번째 까지의 평균 loss : 1.093835961818695
700번째 까지의 평균 loss : 1.0721568727493287
750번째 까지의 평균 loss : 1.0848414015769958
800번째 까지의 평균 loss : 1.0715471112728119
850번째 까지의 평균 loss : 1.0512329137325287
900번째 까지의 평균 loss : 1.1048396611213684
950번째 까지의 평균 loss : 1.055117859840393

  Average training loss: 1.08

검증 중
  Accuracy: 0.5445

Training complete!
